In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
!pip install pyspark


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.



In [3]:
!bash install_spark.sh


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

bash: install_spark.sh: No such file or directory


In [4]:
# Import needed packages 

import pandas as pd
from bs4 import BeautifulSoup
import requests
from requests import get
import json
from io import StringIO
import re
import os
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from datetime import datetime
import csv

In [5]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/project/spark-3.2.1-bin-hadoop3.2"

In [6]:
spark = SparkSession \
    .builder \
    .appName("PySpark App") \
    .config("spark.jars", "postgresql-42.3.2.jar") \
    .getOrCreate()

I noticed that there is a "download" button right above the data table, so those data can be accessed through the API.

In [7]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36'}

In [8]:
def get_response(stock):
    
    '''
    Use the API to get the content of html based on time series
    '''

    stock_url = "https://query1.finance.yahoo.com/v7/finance/download/{}?"
    
    params ={ 
    'period1' : '1334707200',
    'period2' : '1650240000',
    'interval':  '1wk',
    'events': 'history'}
    
    response = requests.get(stock_url.format(stock), params= params, headers=header)
    
    return response

uga = get_response("UGA")
shell = get_response("SHEL.L")

In [9]:
def collect_data(response):
    file = StringIO(response.text)
    reader = csv.reader(file)
    data = list(reader)
    return data

uga_info = collect_data(uga)
shell_info = collect_data(shell)

In [10]:
len(uga_info)
len(shell_info)

523

524

In [11]:
uga_info[:3]
shell_info[:3]

[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'],
 ['2012-04-16',
  '55.930000',
  '56.150002',
  '54.439999',
  '54.849998',
  '54.849998',
  '517500'],
 ['2012-04-23',
  '54.200001',
  '55.360001',
  '54.080002',
  '55.119999',
  '55.119999',
  '412200']]

[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'],
 ['2012-04-16',
  '2158.000000',
  '2167.000000',
  '2082.500000',
  '2136.500000',
  '1212.857422',
  '9858753'],
 ['2012-04-23',
  '2121.000000',
  '2202.000000',
  '2085.500000',
  '2188.000000',
  '1242.093140',
  '20392958']]

In [12]:
def new_fun(values):
    dic = dict(zip(uga_info[0], values))
    return dic

uga_values = [uga_info[1:][i] for i in range(522)]
rename = map(new_fun, uga_values)
uga_stockprice = pd.DataFrame(rename)
uga_stockprice.rename(columns = {'Adj Close':'Adj_Close'}, inplace=True)
uga_stockprice.head()

,Date,Open,High,Low,Close,Adj_Close,Volume
0,2012-04-16,55.930000,56.150002,54.439999,54.849998,54.849998,517500
1,2012-04-23,54.200001,55.360001,54.080002,55.119999,55.119999,412200
2,2012-04-30,55.020000,55.160000,51.770000,52.259998,52.259998,694200
3,2012-05-07,52.080002,53.139999,51.570000,52.750000,52.750000,533400
4,2012-05-14,51.970001,52.200001,50.419998,50.549999,50.549999,231800


In [13]:
def new_fun(values):
    dic = dict(zip(shell_info[0], values))
    return dic
shell_values = [shell_info[1:][i] for i in range(522)]
rename = map(new_fun, shell_values)
shell_stockprice = pd.DataFrame(rename)
shell_stockprice.rename(columns = {'Adj Close':'Adj_Close'}, inplace=True)
shell_stockprice

,Date,Open,High,Low,Close,Adj_Close,Volume
0,2012-04-16,2158.000000,2167.000000,2082.500000,2136.500000,1212.857422,9858753
1,2012-04-23,2121.000000,2202.000000,2085.500000,2188.000000,1242.093140,20392958
2,2012-04-30,2186.000000,2220.214111,2132.000000,2142.000000,1215.979736,13495471
3,2012-05-07,2142.000000,2152.500000,2018.500000,2067.000000,1173.403320,15040906
4,2012-05-14,2051.500000,2051.500000,1956.500000,1982.000000,1140.421631,18347443
...,...,...,...,...,...,...,...
517,2022-03-14,1955.400024,1965.599976,1871.199951,1941.800049,1941.800049,146845246
518,2022-03-21,1959.599976,2122.500000,1954.199951,2110.500000,2110.500000,152968051
519,2022-03-28,2079.000000,2121.500000,2009.000000,2119.500000,2119.500000,121966684
520,2022-04-04,2120.000000,2168.500000,2078.000000,2168.000000,2168.000000,144486519


In [14]:
# convert the data frame into spark data frame
uga_stockprice = spark.createDataFrame(uga_stockprice)
shell_stockprice = spark.createDataFrame(shell_stockprice)

In [15]:
uga_stockprice.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj_Close: string (nullable = true)
 |-- Volume: string (nullable = true)



In [16]:
shell_stockprice.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj_Close: string (nullable = true)
 |-- Volume: string (nullable = true)



In [17]:
uga_stockprice.write.parquet("/project/DataEngineering/parquet_files/uga_info.parquet", mode = 'overwrite')
shell_stockprice.write.parquet("/project/DataEngineering/parquet_files/shell_info.parquet", mode = 'overwrite')

Now look at the fuel price in the US by web scraping from website.

In [18]:
us_url = 'https://www.eia.gov/dnav/pet/hist/LeafHandler.ashx?n=PET&s=EMM_EPM0U_PTE_NUS_DPG&f=W'
r = requests.get(us_url)
r.status_code
soup= BeautifulSoup(r.text, 'lxml')
r.text

200

"<!DOCTYPE HTML PUBLIC '-//W3C//DTD HTML 4.01 Transitional//EN'>\r <html>\r <head>\r <meta http-equiv='X-UA-Compatible' content='IE=9' />\r <script src='../includes/TableFloaterTitle.js'></script>\r <title>Weekly U.S. All Grades Conventional Retail Gasoline Prices (Dollars per Gallon)</title>\r <link rel='StyleSheet' href='../Styles/Pet_wrapper3.css' TYPE='text/css'>\r <link rel='StyleSheet' href='../Styles/leaf_new2.css' TYPE='text/css'>\r <link rel='StyleSheet' href='/styles/Eia_sitewideF.css' type='text/css' />\r <!-- Header Script -->\r <script language='JavaScript' src='/styles/eia_header.js' type='text/javascript'></script>\r <!--/ Header Script -->\r <script src='/global/includes/dnavs/leaf_handler.cfm' type='text/javascript'></script>\r <!-- Footer Script -->\r <script language='JavaScript' src='/styles/eia_footer.js' type='text/javascript'></script>\r <!--/ Footer Script -->\r </head>\r <body>\r <script language='JavaScript' type='text/javascript'>InsertEIAHeaderCode();</scrip

In [19]:
table = soup.find('table', {'class' : "FloatTitle"})

row_data=[]
for row in table.find('tbody').find_all('tr')[1:]:
    data = row.find_all('td')
    col = [td.text.strip() for td in data]
    row_data.append(col)

row_data

[['1994-Dec',
  '12/05',
  '1.143',
  '12/12',
  '1.118',
  '12/19',
  '1.099',
  '12/26',
  '1.088',
  '',
  ''],
 [''],
 ['1995-Jan',
  '01/02',
  '1.104',
  '01/09',
  '1.111',
  '01/16',
  '1.102',
  '01/23',
  '1.110',
  '01/30',
  '1.109'],
 ['1995-Feb',
  '02/06',
  '1.103',
  '02/13',
  '1.099',
  '02/20',
  '1.093',
  '02/27',
  '1.101',
  '',
  ''],
 ['1995-Mar',
  '03/06',
  '1.103',
  '03/13',
  '1.096',
  '03/20',
  '1.095',
  '03/27',
  '1.102',
  '',
  ''],
 ['1995-Apr',
  '04/03',
  '1.116',
  '04/10',
  '1.134',
  '04/17',
  '1.149',
  '04/24',
  '1.173',
  '',
  ''],
 ['1995-May',
  '05/01',
  '1.181',
  '05/08',
  '1.204',
  '05/15',
  '1.213',
  '05/22',
  '1.232',
  '05/29',
  '1.234'],
 ['1995-Jun',
  '06/05',
  '1.234',
  '06/12',
  '1.230',
  '06/19',
  '1.224',
  '06/26',
  '1.217',
  '',
  ''],
 ['1995-Jul',
  '07/03',
  '1.209',
  '07/10',
  '1.200',
  '07/17',
  '1.189',
  '07/24',
  '1.179',
  '07/31',
  '1.166'],
 ['1995-Aug',
  '08/07',
  '1.164',
  '08/1

In [20]:
len(row_data)

357

In [21]:
for i in row_data:
    if len(i) < 10:
        row_data.remove(i)
len(row_data)

329

In [22]:
for i in row_data:
    i[1] = i[0][:4]+ '/' + i[1]
    i[3] = i[0][:4]+ '/' + i[3]
    i[5] = i[0][:4]+ '/' + i[5]
    i[7] = i[0][:4]+ '/' + i[7]
    i[9] = i[0][:4]+ '/' + i[9] 
df = pd.DataFrame(row_data)

In [23]:
df

,0,1,2,3,4,5,6,7,8,9,10
0,1994-Dec,1994/12/05,1.143,1994/12/12,1.118,1994/12/19,1.099,1994/12/26,1.088,1994/,
1,1995-Jan,1995/01/02,1.104,1995/01/09,1.111,1995/01/16,1.102,1995/01/23,1.110,1995/01/30,1.109
2,1995-Feb,1995/02/06,1.103,1995/02/13,1.099,1995/02/20,1.093,1995/02/27,1.101,1995/,
3,1995-Mar,1995/03/06,1.103,1995/03/13,1.096,1995/03/20,1.095,1995/03/27,1.102,1995/,
4,1995-Apr,1995/04/03,1.116,1995/04/10,1.134,1995/04/17,1.149,1995/04/24,1.173,1995/,
...,...,...,...,...,...,...,...,...,...,...,...
324,2021-Dec,2021/12/06,3.279,2021/12/13,3.252,2021/12/20,3.229,2021/12/27,3.211,2021/,
325,2022-Jan,2022/01/03,3.216,2022/01/10,3.238,2022/01/17,3.255,2022/01/24,3.271,2022/01/31,3.321
326,2022-Feb,2022/02/07,3.401,2022/02/14,3.441,2022/02/21,3.480,2022/02/28,3.554,2022/,
327,2022-Mar,2022/03/07,4.031,2022/03/14,4.252,2022/03/21,4.165,2022/03/28,4.152,2022/,


In [24]:
dictionary = dict(zip(df[1], df[2]))
i=3

while i <=9:
    diction = dict(zip(df[i], df[i+1]))
    dictionary.update(diction)
    i+=2

In [25]:
us_fuelprice = pd.DataFrame(list(dictionary.items()),columns = ['Date','Price'] )

In [26]:
us_fuelprice
us_fuelprice.drop( (us_fuelprice[us_fuelprice['Price'].map(len) < 1]).index, inplace=True)

# Sort the dataframe by time
us_fuelprice.sort_values(by='Date', inplace= True)

,Date,Price
0,1994/12/05,1.143
1,1995/01/02,1.104
2,1995/02/06,1.103
3,1995/03/06,1.103
4,1995/04/03,1.116
...,...,...
1453,2021/03/29,2.839
1454,2021/05/31,2.997
1455,2021/08/30,3.118
1456,2021/11/29,3.326


In [27]:
us_fuelprice

,Date,Price
0,1994/12/05,1.143
329,1994/12/12,1.118
658,1994/12/19,1.099
987,1994/12/26,1.088
1,1995/01/02,1.104
...,...,...
985,2022/03/21,4.165
1314,2022/03/28,4.152
328,2022/04/04,4.096
657,2022/04/11,4.019


In [28]:
# Deal with missing values
for i in us_fuelprice['Price']:
    if len(i) <1:
        us_fuelprice = us_fuelprice.dropna(axis=0)

In [29]:
us_fuelprice.reset_index(drop = True, inplace = True)

In [30]:
us_fuelprice_df = spark.createDataFrame(us_fuelprice)
us_fuelprice_df.write.parquet("/project/DataEngineering/parquet_files/us_fuelprice.parquet", mode = 'overwrite')
us_fuelprice_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Price: string (nullable = true)



In [31]:
us_fuelprice

,Date,Price
0,1994/12/05,1.143
1,1994/12/12,1.118
2,1994/12/19,1.099
3,1994/12/26,1.088
4,1995/01/02,1.104
...,...,...
1424,2022/03/21,4.165
1425,2022/03/28,4.152
1426,2022/04/04,4.096
1427,2022/04/11,4.019


Since avaiable dataset can be downloaded through gov.uk, the excel file has be uploaded.

In [32]:
#import excel file about the fuel price in UK
file = pd.read_excel(r'/project/UK_Weekly_Fuel_Prices.xlsx', sheet_name = 'All years')

In [33]:
file.head(10)

,Weekly Prices time series (from 2003),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,ULSP = Ultra low sulphur unleaded petrol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ULSD = Ultra low sulphur diesel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Duty rates over time can be found in the duty ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Empty cells represent weeks where differences ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Freeze panes are turned on. To turn off freeze...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Date,ULSP: Pump price (p/litre),ULSP: Diff on previous WEEK (p/litre),ULSP: Diff on previous YEAR (p/litre),Duty rate ULSP (p/litre),VAT (% rate) ULSP,ULSD: Pump price (p/litre),ULSD: Diff on previous WEEK (p/litre),ULSD: Diff on previous YEAR (p/litre),Duty rate ULSD (p/litre),VAT (% rate) ULSD,NaN,NaN,NaN,NaN,NaN,NaN
6,2003-06-09 00:00:00,74.590284,NaN,NaN,45.82,17.5,76.773394,NaN,NaN,45.82,17.5,NaN,NaN,NaN,NaN,NaN,NaN
7,2003-06-16 00:00:00,74.469143,-0.121141,NaN,45.82,17.5,76.689054,-0.08434,NaN,45.82,17.5,NaN,NaN,NaN,NaN,NaN,NaN
8,2003-06-23 00:00:00,74.423566,-0.045577,NaN,45.82,17.5,76.620546,-0.068508,NaN,45.82,17.5,NaN,NaN,NaN,NaN,NaN,NaN
9,2003-06-30 00:00:00,74.352421,-0.071145,NaN,45.82,17.5,76.50526,-0.115286,NaN,45.82,17.5,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
file.columns =file.iloc[5]
file = file.iloc[6: , :]

In [35]:
file

5,Date,ULSP: Pump price (p/litre),ULSP: Diff on previous WEEK (p/litre),ULSP: Diff on previous YEAR (p/litre),Duty rate ULSP (p/litre),VAT (% rate) ULSP,ULSD: Pump price (p/litre),ULSD: Diff on previous WEEK (p/litre),ULSD: Diff on previous YEAR (p/litre),Duty rate ULSD (p/litre),VAT (% rate) ULSD,NaN,NaN,NaN,NaN,NaN,NaN
6,2003-06-09 00:00:00,74.590284,NaN,NaN,45.82,17.5,76.773394,NaN,NaN,45.82,17.5,NaN,NaN,NaN,NaN,NaN,NaN
7,2003-06-16 00:00:00,74.469143,-0.121141,NaN,45.82,17.5,76.689054,-0.08434,NaN,45.82,17.5,NaN,NaN,NaN,NaN,NaN,NaN
8,2003-06-23 00:00:00,74.423566,-0.045577,NaN,45.82,17.5,76.620546,-0.068508,NaN,45.82,17.5,NaN,NaN,NaN,NaN,NaN,NaN
9,2003-06-30 00:00:00,74.352421,-0.071145,NaN,45.82,17.5,76.50526,-0.115286,NaN,45.82,17.5,NaN,NaN,NaN,NaN,NaN,NaN
10,2003-07-07 00:00:00,74.283737,-0.068684,NaN,45.82,17.5,76.457376,-0.047884,NaN,45.82,17.5,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,2022-03-14 00:00:00,159.96,7.01,36.059117,57.95,20,169.48,10.92,41.236275,57.95,20,NaN,NaN,NaN,NaN,NaN,NaN
986,2022-03-21 00:00:00,165.37,5.41,40.801753,57.95,20,177.47,7.99,48.450879,57.95,20,NaN,NaN,NaN,NaN,NaN,NaN
987,2022-03-28 00:00:00,162.651769,-2.718231,37.524793,52.95,20,176.442403,-1.027597,47.127295,52.95,20,NaN,NaN,NaN,NaN,NaN,NaN
988,2022-04-04 00:00:00,161.91,-0.741769,36.669577,52.95,20,176,-0.442403,46.649255,52.95,20,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
from datetime import datetime

file['Date'] = [i.date() for i in file['Date']]

In [37]:
uk_fuelprice = file.iloc[: ,  :3]
uk_fuelprice.columns = ["Date", 'uk_price', 'uk_diff']

In [38]:
uk_fuelprice['uk_diff'] =[float(i) for i in uk_fuelprice['uk_diff'] ]
uk_fuelprice['uk_diff'].round(decimals = 3)

6        NaN
7     -0.121
8     -0.046
9     -0.071
10    -0.069
       ...  
985    7.010
986    5.410
987   -2.718
988   -0.742
989   -0.130
Name: uk_diff, Length: 984, dtype: float64

In [39]:
uk_fuelprice_df = spark.createDataFrame(uk_fuelprice)
uk_fuelprice_df.printSchema()
uk_fuelprice_df.write.parquet("/project/DataEngineering/parquet_files/uk_fuelprice.parquet", mode= 'overwrite')

root
 |-- Date: date (nullable = true)
 |-- uk_price: double (nullable = true)
 |-- uk_diff: double (nullable = true)



In [40]:
uga = spark.read.parquet("/project/DataEngineering/parquet_files/uga_info.parquet").toPandas()

In [41]:
uga.dtypes

Date         object
Open         object
High         object
Low          object
Close        object
Adj_Close    object
Volume       object
dtype: object

In [42]:
uga['High']= uga['High'].astype(float)
uga['Low']= uga['Low'].astype(float)
uga['Close']= uga['Close'].astype(float)
uga['Volume']= uga['Volume'].astype(float)

In [43]:
uga['uga_Difference'] = (uga['High'] - uga['Low'])/uga['Close']*100
uga.drop(labels= ['High', 'Low', 'Open', 'Adj_Close'], axis=1, inplace=True)

In [44]:
uga = uga.rename(columns={"Close":"uga_Close", "Volume":"uga_Volume"})
uga

,Date,uga_Close,uga_Volume,uga_Difference
0,2019-10-07,30.459999,37400.0,5.712410
1,2019-10-14,30.000000,31500.0,2.100003
2,2019-10-21,31.139999,37400.0,5.780350
3,2019-10-28,31.540001,50700.0,5.009512
4,2019-11-04,31.170000,49200.0,5.646458
...,...,...,...,...
517,2019-09-02,28.820000,76200.0,8.501037
518,2019-09-09,28.510000,41200.0,6.453879
519,2019-09-16,30.670000,151100.0,8.444734
520,2019-09-23,29.790001,53200.0,4.867402


In [45]:
shell = spark.read.parquet("/project/DataEngineering/parquet_files/shell_info.parquet").toPandas()
shell

,Date,Open,High,Low,Close,Adj_Close,Volume
0,2019-10-07,2317.500000,2347.000000,2278.500000,2290.500000,2046.361816,43462714
1,2019-10-14,2299.000000,2333.489990,2235.500000,2240.000000,2001.244629,40636681
2,2019-10-21,2235.000000,2332.500000,26.087999,2325.500000,2077.630859,28698016
3,2019-10-28,2330.500000,2345.000000,2233.000000,2260.500000,2019.559204,40717990
4,2019-11-04,2269.500000,2356.000000,3.045000,2341.000000,2091.479248,59819909
...,...,...,...,...,...,...,...
517,2019-09-02,2281.500000,2308.500000,2246.000000,2259.500000,2018.665649,32084357
518,2019-09-09,2279.000000,2331.500000,2261.000000,2282.500000,2039.214111,32091732
519,2019-09-16,2356.000000,2373.000000,2310.500000,2343.500000,2093.712402,53935799
520,2019-09-23,2353.500000,2414.500000,2311.500000,2408.500000,2151.784424,45485098


In [46]:
shell.dtypes

Date         object
Open         object
High         object
Low          object
Close        object
Adj_Close    object
Volume       object
dtype: object

In [47]:
shell['High']= shell['High'].astype(float)
shell['Low']= shell['Low'].astype(float)
shell['Close']= shell['Close'].astype(float)
shell['Volume']= shell['Volume'].astype(float)

In [48]:
shell['shell_Difference'] = (shell['High'] - shell['Low'])/shell['Close']*100
shell.drop(labels= ['High', 'Low', 'Open', 'Adj_Close'], axis=1, inplace=True)
shell = shell.rename(columns={"Close":"shell_Close", "Volume":"shell_Volume"})

In [49]:
shell.head()

,Date,shell_Close,shell_Volume,shell_Difference
0,2019-10-07,2290.5,43462714.0,2.990613
1,2019-10-14,2240.0,40636681.0,4.374553
2,2019-10-21,2325.5,28698016.0,99.179187
3,2019-10-28,2260.5,40717990.0,4.954656
4,2019-11-04,2341.0,59819909.0,100.510679


In [50]:
stock_price = pd.merge(uga, shell, on='Date')

In [51]:
stock_price

,Date,uga_Close,uga_Volume,uga_Difference,shell_Close,shell_Volume,shell_Difference
0,2019-10-07,30.459999,37400.0,5.712410,2290.5,43462714.0,2.990613
1,2019-10-14,30.000000,31500.0,2.100003,2240.0,40636681.0,4.374553
2,2019-10-21,31.139999,37400.0,5.780350,2325.5,28698016.0,99.179187
3,2019-10-28,31.540001,50700.0,5.009512,2260.5,40717990.0,4.954656
4,2019-11-04,31.170000,49200.0,5.646458,2341.0,59819909.0,100.510679
...,...,...,...,...,...,...,...
517,2019-09-02,28.820000,76200.0,8.501037,2259.5,32084357.0,2.766099
518,2019-09-09,28.510000,41200.0,6.453879,2282.5,32091732.0,3.088719
519,2019-09-16,30.670000,151100.0,8.444734,2343.5,53935799.0,2.666951
520,2019-09-23,29.790001,53200.0,4.867402,2408.5,45485098.0,4.276521


In [52]:
stock_price = spark.createDataFrame(stock_price)
stock_price.printSchema()
stock_price.write.parquet("/project/DataEngineering/parquet_files/stock_price.parquet", mode ='overwrite')

root
 |-- Date: string (nullable = true)
 |-- uga_Close: double (nullable = true)
 |-- uga_Volume: double (nullable = true)
 |-- uga_Difference: double (nullable = true)
 |-- shell_Close: double (nullable = true)
 |-- shell_Volume: double (nullable = true)
 |-- shell_Difference: double (nullable = true)



We already have the stock price related data of UGA and Shell, the prices of fuel in the UK and America are supposed to be taken account it. Consequently, the price will be web scraped from govenment websites.

Since we want to compare the fuel price and stock price later, the date range should be uniformed. In this case, a part of data will be selected as a new data table.

In [53]:
us_fuelprice.sort_values(by='Date', inplace= True)
us_fuelprice.reset_index(drop = True, inplace=True)

# Get the position of where we should start
us_fuelprice.loc[us_fuelprice['Date'] == "2012/04/16"]

,Date,Price
906,2012/04/16,3.923


In [54]:
us_price= us_fuelprice[906:-1]
us_price.reset_index( drop = True, inplace=True)
us_price['Price'] = pd.to_numeric(us_price['Price'])
us_price

<ipython-input-54-3355b2f5798a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_price['Price'] = pd.to_numeric(us_price['Price'])


,Date,Price
0,2012/04/16,3.923
1,2012/04/23,3.861
2,2012/04/30,3.821
3,2012/05/07,3.773
4,2012/05/14,3.713
...,...,...
517,2022/03/14,4.252
518,2022/03/21,4.165
519,2022/03/28,4.152
520,2022/04/04,4.096


In [55]:
difference = []

i=520
while i >= 0:
    diff = us_price['Price'].iloc[i+1] -us_price['Price'].iloc[i]
    difference.append(diff)
    i -= 1

difference = difference[ : : -1]    
difference.append(-0.027)

In [56]:
us_price['us_price_diff'] = difference
us_price['us_price_diff'].round(decimals = 3)
us_price.columns = ['Date', 'us_price', 'us_price_diff']

<ipython-input-56-2fafb38ac925>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_price['us_price_diff'] = difference


0     -0.062
1     -0.040
2     -0.048
3     -0.060
4     -0.037
       ...  
517   -0.087
518   -0.013
519   -0.056
520   -0.077
521   -0.027
Name: us_price_diff, Length: 522, dtype: float64

In [57]:
us_price

,Date,us_price,us_price_diff
0,2012/04/16,3.923,-0.062
1,2012/04/23,3.861,-0.040
2,2012/04/30,3.821,-0.048
3,2012/05/07,3.773,-0.060
4,2012/05/14,3.713,-0.037
...,...,...,...
517,2022/03/14,4.252,-0.087
518,2022/03/21,4.165,-0.013
519,2022/03/28,4.152,-0.056
520,2022/04/04,4.096,-0.077


In [58]:
us_price['Date'] = [i.replace("/", '-') for i in us_price['Date']]
us_price

<ipython-input-58-b5ff215a1cd3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_price['Date'] = [i.replace("/", '-') for i in us_price['Date']]


,Date,us_price,us_price_diff
0,2012-04-16,3.923,-0.062
1,2012-04-23,3.861,-0.040
2,2012-04-30,3.821,-0.048
3,2012-05-07,3.773,-0.060
4,2012-05-14,3.713,-0.037
...,...,...,...
517,2022-03-14,4.252,-0.087
518,2022-03-21,4.165,-0.013
519,2022-03-28,4.152,-0.056
520,2022-04-04,4.096,-0.077


In [59]:
uk_fuelprice=spark.read.parquet("/project/DataEngineering/parquet_files/uk_fuelprice.parquet").toPandas()
uk_fuelprice

,Date,uk_price,uk_diff
0,2008-02-25,104.648274,0.952977
1,2008-03-03,105.238253,0.589979
2,2008-03-10,105.957288,0.719035
3,2008-03-17,106.414532,0.457244
4,2008-03-24,106.904702,0.490170
...,...,...,...
979,2022-03-14,159.960000,7.010000
980,2022-03-21,165.370000,5.410000
981,2022-03-28,162.651769,-2.718231
982,2022-04-04,161.910000,-0.741769


In [60]:
# Sort the dataframe by time
uk_fuelprice.sort_values(by='Date', inplace= True)
uk_fuelprice.reset_index(drop = True, inplace= True)

In [61]:
timestampStr = [i.strftime("%Y/%m/%d") for i in uk_fuelprice['Date']]

In [62]:
uk_fuelprice['Date'] = timestampStr

uk_fuelprice[uk_fuelprice['Date'] == '2012/04/16']

uk_price = uk_fuelprice[462:].reset_index( drop=True)
uk_price['uk_diff'] = uk_price['uk_diff'].round(decimals = 3)
uk_price['Date'] = [i.replace("/", '-') for i in uk_price['Date']]
uk_price

,Date,uk_price,uk_diff
462,2012/04/16,142.172754,0.20121


,Date,uk_price,uk_diff
0,2012-04-16,142.172754,0.201
1,2012-04-23,141.759020,-0.414
2,2012-04-30,141.004212,-0.755
3,2012-05-07,140.136502,-0.868
4,2012-05-14,138.286574,-1.850
...,...,...,...
517,2022-03-14,159.960000,7.010
518,2022-03-21,165.370000,5.410
519,2022-03-28,162.651769,-2.718
520,2022-04-04,161.910000,-0.742


In [63]:
fuel_price = pd.merge(us_price, uk_price, on='Date')

In [64]:
fuel_price = spark.createDataFrame(fuel_price)
fuel_price.printSchema()
fuel_price.write.parquet("/project/DataEngineering/parquet_files/fuel_price.parquet", mode = "overwrite")

root
 |-- Date: string (nullable = true)
 |-- us_price: double (nullable = true)
 |-- us_price_diff: double (nullable = true)
 |-- uk_price: double (nullable = true)
 |-- uk_diff: double (nullable = true)



In [65]:
new= pd.merge(us_price, uga, on='Date')
us_diff = new[['Date', 'us_price_diff', 'uga_Difference']]
us_diff = spark.createDataFrame(us_diff)
us_diff.printSchema()
us_diff.write.parquet("/project/DataEngineering/parquet_files/us_diff.parquet", mode='overwrite')

root
 |-- Date: string (nullable = true)
 |-- us_price_diff: double (nullable = true)
 |-- uga_Difference: double (nullable = true)



In [66]:
us_info = new[['Date', 'uga_Close', 'uga_Volume', 'uga_Difference', 'us_price']]
us_info =  spark.createDataFrame(us_info)
us_info.printSchema()
us_info.write.parquet("/project/DataEngineering/parquet_files/us_info.parquet", mode='overwrite')

root
 |-- Date: string (nullable = true)
 |-- uga_Close: double (nullable = true)
 |-- uga_Volume: double (nullable = true)
 |-- uga_Difference: double (nullable = true)
 |-- us_price: double (nullable = true)



In [67]:
new2= pd.merge(uk_price, shell, on='Date')
uk_diff = new2[['Date', 'uk_diff', 'shell_Difference']]
uk_diff = spark.createDataFrame(uk_diff)
uk_diff.printSchema()
uk_diff.write.parquet("/project/DataEngineering/parquet_files/uk_diff.parquet", mode='overwrite')

root
 |-- Date: string (nullable = true)
 |-- uk_diff: double (nullable = true)
 |-- shell_Difference: double (nullable = true)



In [68]:
uk_info = new2[['Date', 'shell_Close', 'shell_Volume', 'shell_Difference', 'uk_price']]
uk_info = spark.createDataFrame(uk_info)
uk_info.printSchema()
uk_info.write.parquet("/project/DataEngineering/parquet_files/uk_info.parquet", mode='overwrite')

root
 |-- Date: string (nullable = true)
 |-- shell_Close: double (nullable = true)
 |-- shell_Volume: double (nullable = true)
 |-- shell_Difference: double (nullable = true)
 |-- uk_price: double (nullable = true)



In [72]:
!PGPASSWORD=qwerty123 psql -h depgdb.crhso94tou3n.eu-west-2.rds.amazonaws.com -d huizihu21 -U huizihu21 -c '\i Schema.sql'



         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

DROP SCHEMA
CREATE SCHEMA
psql:Schema.sql:6: NOTICE:  table "stock_price" does not exist, skipping
DROP TABLE
psql:Schema.sql:7: NOTICE:  table "fuel_price" does not exist, skipping
DROP TABLE
psql:Schema.sql:8: NOTICE:  table "us_info" does not exist, skipping
DROP TABLE
psql:Schema.sql:9: NOTICE:  table "uk_info" does not exis

In [70]:
postgres_uri = "jdbc:postgresql://depgdb.crhso94tou3n.eu-west-2.rds.amazonaws.com:5432/huizihu21"
user = "huizihu21"
password = "qwerty123"

In [71]:
stock_price.write.jdbc(url=postgres_uri, table="fuel_and_stock_price.stock_price", mode="append", properties={"user":user, "password": password, "driver": "org.postgresql.Driver" })

fuel_price.write.jdbc(url=postgres_uri, table="fuel_and_stock_price.fuel_price", mode="append", properties={"user":user, "password": password, "driver": "org.postgresql.Driver" })

us_info.write.jdbc(url=postgres_uri, table="fuel_and_stock_price.us_info", mode="append", properties={"user":user, "password": password, "driver": "org.postgresql.Driver" })

uk_info.write.jdbc(url=postgres_uri, table="fuel_and_stock_price.uk_info", mode="append", properties={"user":user, "password": password, "driver": "org.postgresql.Driver" })

us_diff.write.jdbc(url=postgres_uri, table="fuel_and_stock_price.us_diff", mode="append", properties={"user":user, "password": password, "driver": "org.postgresql.Driver" })

uk_diff.write.jdbc(url=postgres_uri, table="fuel_and_stock_price.uk_diff", mode="append", properties={"user":user, "password": password, "driver": "org.postgresql.Driver" })


Py4JJavaError: An error occurred while calling o273.jdbc.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.net.URLClassLoader.findClass(URLClassLoader.java:387)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:101)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:218)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:222)
	at org.apache.spark.sql.execution.d